In [62]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

internal_urls = set()
external_urls = set()

def is_valid(url):
    parsed = urlparse(url)
    
    if parsed.scheme.startswith('http') and bool(parsed.netloc):
        return True
    else:
        return False

def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    total_internal = 0
    total_external = 0
        
    for a_tag in soup.findAll("a"):
        
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
            
        if href in internal_urls:
            total_internal = total_internal + 1
            # already in the set
            continue
            
        if domain_name not in href:
            total_external = total_external + 1
            # external link
            if href not in external_urls:
#                 print(f"[!] External link: {href}")
                external_urls.add(href)
            continue
            
#         print(f"[*] Internal link: {href}")
        urls.add(href)
        internal_urls.add(href)
        total_internal = total_internal + 1
        
#     print("[+] Total Internal links:", len(internal_urls))
#     print("[+] Total External links:", len(external_urls))
    
    print("[+] Total Internal links:", total_internal)
    print("[+] Total External links:", total_external)

get_all_website_links('https://www.bmn.nl/')

[+] Total Internal links: 563
[+] Total External links: 5
